In [78]:
! pip install chromadb langchain tabulate google-api-python-client openai python-dotenv --quiet

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


# INTRODUCTION
<a target="_blank" href="https://colab.research.google.com/github/ejcv/NLP_course/blob/main/third_session_nlp_course.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

## LLMs 🦜
After the revolution the attention mechanism and the transformer architecture brought, the next big thing in NLP was the introduction of language models. Language models are models that are trained to predict the next word in a sequence of words. The idea is that if a model can predict the next word in a sequence, it must have learned something about the language and the structure of the text. The first language model was the GPT model, which was trained on a huge dataset, they have existed for a while but it was until the launch of ChatGPT that its popularity exploded.

LLMs are a very powerful tool, they can be used for a variety of tasks, such as text generation, text classification, text summarization, and many more. But its power resides when we embed it into our applications. For example, we can use a language model to generate text for a chatbot, or we can use it to classify the sentiment of a text, or we can use it to summarize a text. The possibilities are endless.



There are several ways to interact with a LLM, you can use it locally, which is hard (it is getting easier and easier though) or you can use it on a cloud service. Once you have that, you can either implement all the abstractions you need to make it more useful, or you can use a framework that handles all of that for you. We will explore both ways.

## OPENAI API

In [1]:
# we can interact with the llm via the openai api, or its equivalent in google

import openai
import os
from dotenv import load_dotenv

load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

In [2]:
chat_completion = openai.ChatCompletion.create(model="gpt-3.5-turbo", messages=[{"role": "user", "content": "Hello world"}])
print(chat_completion.choices[0])

{
  "index": 0,
  "message": {
    "role": "assistant",
    "content": "Hello! How can I assist you today?"
  },
  "finish_reason": "stop"
}


In [3]:
# lets load our data and see how it performs classyfing the data
# we will use the same data as in the previous session
import pandas as pd
df = pd.read_csv('datasets/tweet_emotions.csv')
df.head()

,tweet_id,sentiment,content
0,1956967341,empty,@tiffanylue i know i was listenin to bad habi...
1,1956967666,sadness,Layin n bed with a headache ughhhh...waitin o...
2,1956967696,sadness,Funeral ceremony...gloomy friday...
3,1956967789,enthusiasm,wants to hang out with friends SOON!
4,1956968416,neutral,@dannycastillo We want to trade with someone w...


In [4]:
sentiments = df['sentiment'].unique()
sentiments

array(['empty', 'sadness', 'enthusiasm', 'neutral', 'worry', 'surprise',
       'love', 'fun', 'hate', 'happiness', 'boredom', 'relief', 'anger'],
      dtype=object)

## Zero Shot learning 🥚
One of the cool things of LLMs are the emergent abilities, for example a model may not have been trained to do a certain task, but it still can do it. This is the case of few shot learning, where a model can do a task without any prior examples.

## Few shot learning 🐥
Similarly, few shot learning is the ability of a model to do a task with a few examples. For example, if we want to do sentiment analysis, we can give the model a few examples of positive and negative sentences, and it will be able to do sentiment analysis on new sentences.

In [5]:
# Thats exactly what we will do

# Lets create a prompt for the system

system_prompt = f"""
You are a sentiment classifier specialiazed assistant. Your job is to classify the sentiment of tweets.

I will pass you a tweet and you will give me the corresponding sentiment.

The valid sentiments are {", ".join(sentiments)}.
"""

print(system_prompt)


You are a sentiment classifier specialiazed assistant. Your job is to classify the sentiment of tweets.

I will pass you a tweet and you will give me the corresponding sentiment.

The valid sentiments are empty, sadness, enthusiasm, neutral, worry, surprise, love, fun, hate, happiness, boredom, relief, anger.



In [6]:
def classify_sentiment(text:str) -> str:
    response = openai.ChatCompletion.create(model="gpt-3.5-turbo", messages = [{
      "role": "system", 
      "content": system_prompt  
    },
    {
      "role": "user",
      "content": text
      }])

    return response.choices[0]['message']['content']

In [7]:
idx = 45
df.iloc[idx]

tweet_id                                            1956978410
sentiment                                                worry
content      Bed!!!!!... its time,..... hope i go to school...
Name: 45, dtype: object

In [8]:
answer = classify_sentiment(df.iloc[idx]['content'])
print(answer)

worry


In [9]:
# we can change the system prompt to have a few examples on how to classify the tweets
system_prompt = f"""
You are a sentiment classifier specialiazed assistant. Your job is to classify the sentiment of tweets.

I will pass you a tweet and you will give me the corresponding sentiment.

The valid sentiments are: "{", ".join(sentiments)}."

Here are some examples:
- input: {df.iloc[0]['content']}
- your answer: {df.iloc[0]['sentiment']}

- input: {df.iloc[1]['content']}
- your answer: {df.iloc[1]['sentiment']}

- input: {df.iloc[2]['content']}
- your answer: {df.iloc[2]['sentiment']}

"""
print(system_prompt)


You are a sentiment classifier specialiazed assistant. Your job is to classify the sentiment of tweets.

I will pass you a tweet and you will give me the corresponding sentiment.

The valid sentiments are: "empty, sadness, enthusiasm, neutral, worry, surprise, love, fun, hate, happiness, boredom, relief, anger."

Here are some examples:
- input: @tiffanylue i know  i was listenin to bad habit earlier and i started freakin at his part =[
- your answer: empty

- input: Layin n bed with a headache  ughhhh...waitin on your call...
- your answer: sadness

- input: Funeral ceremony...gloomy friday...
- your answer: sadness




In [10]:
idx = 100
df.iloc[idx]['content'], df.iloc[idx]['sentiment']

('First ever dropped call on my mobile. On a call to @Telstra no less! ( being charged for data even though I have a data pack  )',
 'worry')

In [11]:
answer = classify_sentiment(df.iloc[idx]['content'])
print(answer)

worry


## LangChain 🦜⛓
Langchain is a framework that abstracts the process of building an application that interacts with a language model. It is designed to be modular and extensible, allowing developers to easily swap out components for their own implementations.

## Chains
Using an LLM in isolation is fine for simple applications, but more complex applications require chaining LLMs - either with each other or with other components.

LangChain provides the Chain interface for such "chained" applications. We define a Chain very generically as a sequence of calls to components, which can include other chains.

### Vector Stores

In [12]:
from langchain.document_loaders import TextLoader
from langchain.embeddings import HuggingFaceEmbeddings, SentenceTransformerEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma

# Load the document, split it into chunks, embed each chunk and load it into the vector store.
raw_documents = TextLoader('datasets/state_of_the_union.txt').load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
documents = text_splitter.split_documents(raw_documents)
db = Chroma.from_documents(documents, SentenceTransformerEmbeddings())

In [13]:
query = "What did the president say about Ketanji Brown Jackson"
docs = db.similarity_search(query)
print(docs[0].page_content)

Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. 

Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. 

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.


In [14]:
from langchain.chat_models import ChatOpenAI
model_name = "gpt-3.5-turbo"
llm = ChatOpenAI(model_name=model_name)

In [15]:
from langchain.chains.question_answering import load_qa_chain
chain = load_qa_chain(llm, chain_type="stuff",verbose=True)

matching_docs = db.similarity_search(query)
answer =  chain.run(input_documents=matching_docs, question=query)
answer



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the users question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. 

Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. 

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our natio

"The president said that he nominated Circuit Court of Appeals Judge Ketanji Brown Jackson, describing her as one of our nation's top legal minds who will continue Justice Breyer's legacy of excellence."

### Agents

In [16]:
from langchain.agents import create_pandas_dataframe_agent
from langchain.chat_models import ChatOpenAI
from langchain.agents.agent_types import AgentType
from langchain.llms import OpenAI

In [17]:
agent = create_pandas_dataframe_agent(OpenAI(temperature=0), df, verbose=True)

In [18]:
agent = create_pandas_dataframe_agent(
    ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613"),
    df,
    verbose=True,
    agent_type=AgentType.OPENAI_FUNCTIONS,
)

In [19]:
agent.run("how many rows are there?")



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `df.shape[0]`


40000There are 40,000 rows in the dataframe.

> Finished chain.


'There are 40,000 rows in the dataframe.'

In [20]:
agent.run("what are the columns types?")



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `df.dtypes`


tweet_id      int64
sentiment    object
content      object
dtype: objectThe column types in the dataframe are as follows:
- `tweet_id`: int64
- `sentiment`: object
- `content`: object

> Finished chain.


'The column types in the dataframe are as follows:\n- `tweet_id`: int64\n- `sentiment`: object\n- `content`: object'

In [21]:
agent.run("Can you tell me the unique values in the sentiment column?")



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `df['sentiment'].unique().tolist()`


['empty', 'sadness', 'enthusiasm', 'neutral', 'worry', 'surprise', 'love', 'fun', 'hate', 'happiness', 'boredom', 'relief', 'anger']The unique values in the `sentiment` column are: "empty", "sadness", "enthusiasm", "neutral", "worry", "surprise", "love", "fun", "hate", "happiness", "boredom", "relief", and "anger".

> Finished chain.


'The unique values in the `sentiment` column are: "empty", "sadness", "enthusiasm", "neutral", "worry", "surprise", "love", "fun", "hate", "happiness", "boredom", "relief", and "anger".'

### Memory

In [22]:
from langchain.agents import ZeroShotAgent, Tool, AgentExecutor
from langchain.memory import ConversationBufferMemory
from langchain import OpenAI, LLMChain
from langchain.utilities import GoogleSearchAPIWrapper
from langchain.tools.python.tool import PythonREPLTool
from langchain.python import PythonREPL

In [23]:
search = GoogleSearchAPIWrapper()
tools = [
    Tool(
        name="Search",
        func=search.run,
        description="useful for when you need to answer questions about current events",
    ),
    Tool(
        name="PythonREPL",
        func=PythonREPLTool(),
        description="useful for when you need to test a small piece of code",
    )
]

In [24]:
prefix = """Have a conversation with a human, answering the following questions as best you can. You have access to the following tools:"""
suffix = """Begin!"

{chat_history}
Question: {input}
{agent_scratchpad}"""

prompt = ZeroShotAgent.create_prompt(
    tools,
    prefix=prefix,
    suffix=suffix,
    input_variables=["input", "chat_history", "agent_scratchpad"],
)
memory = ConversationBufferMemory(memory_key="chat_history")

In [25]:
llm_chain = LLMChain(llm=OpenAI(temperature=0), prompt=prompt)
agent = ZeroShotAgent(llm_chain=llm_chain, tools=tools, verbose=True)
agent_chain = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True, memory=memory
)

In [26]:
agent_chain.run(input="What is Mexico population?")



> Entering new AgentExecutor chain...
Thought: I need to find out the population of Mexico
Action: Search
Action Input: Mexico population
Observation: Mexico 2023 population is estimated at 128,455,567 people at mid year. Mexico population is equivalent to 1.6% of the total world population. Mexico ranks ... Mexico covers 1,972,550 km2 (761,610 sq mi), making it the world's 13th-largest country by area; with a population of over 126 million, it is the 10th-most- ... Mexico. Demographic data as of July 1, 2023, economic data for 2022 (source) Print Share ... Current and Projected Population. Population, total - Mexico from The World Bank: Data. ... Population and Vital Statistics Reprot ( various years ), ( 5 ) U.S. Census Bureau: International ... Since 1979, the Population Council has worked in Mexico to improve reproductive health through high-quality research, program evaluation, and technical ... Dec 6, 2016 ... Estimates show that the total Mexican population will reach 150 mill

"Mexico's population is estimated at 128,455,567 people as of mid-2023."

In [27]:
agent_chain.run(input="if the growth rate of the population is 2% yearly, what will the population be in 2028 (now is 2023)?")



> Entering new AgentExecutor chain...


Python REPL can execute arbitrary code. Use with caution.


Thought: I need to calculate the population in 2028
Action: PythonREPL
Action Input: 128455667 * 1.02 ** 5
Observation: 
Thought: I now know the final answer
Final Answer: 134,845,945 people

> Finished chain.


'134,845,945 people'

In [28]:
agent_chain.run("What python version are you running?")



> Entering new AgentExecutor chain...
Thought: I need to find out what version of Python I'm running
Action: PythonREPL
Action Input: print(sys.version)
Observation: 3.10.6 | packaged by conda-forge | (main, Aug 22 2022, 20:41:22) [Clang 13.0.1 ]

Thought: I now know the final answer
Final Answer: I am running Python version 3.10.6.

> Finished chain.


'I am running Python version 3.10.6.'